In [ ]:
!pip install -q -U keras-tuner
!pip install wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=192dc226c375923802b4cdc179f1974340fc758c966863d80e1a74daa01ad984
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import string
import datetime
import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Bidirectional, concatenate
from keras.models import Model
from keras.initializers import Constant
from keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('amazon_fakereviewsDS.csv', on_bad_lines='skip')

# Use only 'text_' and 'label' columns
df = df[['text_', 'label']]

# Changing the categorical values from "CG" and "or" to 0 and 1
df['label'] = pd.Categorical(df['label'])
df['label'] = df['label'].cat.codes

def remove_punctuation(txt):
    # We lower-case every word
    text_lower = "".join([c.lower() for c in txt])
    # Then we remove punctuations
    txt_nonpunct = "".join([c for c in text_lower if c not in string.punctuation])
    return txt_nonpunct

# Apply remove_punctuation function to 'text_' column
df['text_'] = df['text_'].apply(lambda x: remove_punctuation(x))

# Split the data in 80:10:10 for train:valid:test dataset
train_size = 0.8
valid_size = 0.1
test_size = 0.1

train_data, temp_data = train_test_split(df, train_size=train_size)
valid_data, test_data = train_test_split(temp_data, train_size=valid_size/(valid_size + test_size))

# Separate the "text_" and "label" columns
X_train = train_data['text_']
y_train = train_data['label']
X_valid = valid_data['text_']
y_valid = valid_data['label']
X_test = test_data['text_']
y_test = test_data['label']

# Tokenization and padding
max_length = 200
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_valid_sequences = tokenizer.texts_to_sequences(X_valid)
X_valid_padded = pad_sequences(X_valid_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')

# Word2Vec embeddings (if needed)
# Set up the hyperparameters for our Word2Vec embedding
W2V_SIZE = 100
W2V_WINDOW = 7
W2V_EPOCH = 64
W2V_MIN_COUNT = 5
W2V_SG = 1
W2V_SEED = 42
W2V_WORKERS = 4

documents = [text.split() for text in X_train]
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=W2V_SIZE,
                                            sg=W2V_SG,
                                            window=W2V_WINDOW,
                                            seed=W2V_SEED,
                                            workers=W2V_WORKERS)
w2v_model.build_vocab(documents)
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

# Embedding matrix
embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

# Define the model architecture
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=W2V_SIZE,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)

input_text = Input(shape=(max_length,), name='text_input')
embeddings = embedding_layer(input_text)
lstm = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(embeddings)
dense = Dense(64, activation='relu')(lstm)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=input_text, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Training the model
history = model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_valid_padded, y_valid))

# Plotting the training and validation metrics
"""plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.show()"""


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 100)          4538300   
                                                                 
 bidirectional (Bidirection  (None, 128)               84480     
 al)                                                             
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4631101 (17.67 MB)
Trainable params: 92801 (362.50 KB)
Non-trainable params: 4538300 (17.31 MB)
_________________

"plt.figure(figsize=(12, 5))\nplt.subplot(1, 2, 1)\nplot_graphs(history, 'accuracy')\nplt.subplot(1, 2, 2)\nplot_graphs(history, 'loss')\nplt.show()"

In [ ]:
from sklearn.metrics import classification_report


# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(X_train_padded, y_train)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Predict the classes for the test data
y_pred = model.predict(X_test_padded)
y_pred_classes = (y_pred > 0.5).astype("int32")

# Generate and print the classification report
report = classification_report(y_test, y_pred_classes, target_names=['-1(fake)', '+1(real)'])
print("Classification Report:\n", report)

1011/1011 [==============================] - 82s 82ms/step - loss: 0.0581 - accuracy: 0.9780
Training Accuracy: 97.80%
127/127 [==============================] - 11s 84ms/step - loss: 0.1107 - accuracy: 0.9632
Test Accuracy: 96.32%
127/127 [==============================] - 10s 80ms/step
Classification Report:
               precision    recall  f1-score   support

    -1(fake)       0.96      0.97      0.96      2013
    +1(real)       0.97      0.96      0.96      2031

    accuracy                           0.96      4044
   macro avg       0.96      0.96      0.96      4044
weighted avg       0.96      0.96      0.96      4044

